# Preliminaries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import scipy

In [2]:
from dissipationtheory.constants import ureg, epsilon0, qe
from dissipationtheory.dissipation9a import CantileverModel, SampleModel1, SampleModel2, SampleModel3
from dissipationtheory.dissipation9b import SampleModel1Jit, SampleModel2Jit, SampleModel3Jit

In [3]:
from dissipationtheory.dissipation13e import KmatrixI_jit, KmatrixII_jit,  KmatrixIII_jit, KmatrixIV_jit

In [4]:
from dissipationtheory.dissipation13e import twodimCobject

In [5]:
THIS = 'dissipation-theory--Study-72--'
figs = {}
obj = {}
results = {}

# Finite-tip calculation

Use a representative cantilever and Type I sample.  
Make the $h/R$ ratio large, so the point-probe approximation is a good one.

In [6]:
cantilever = CantileverModel(
    f_c = ureg.Quantity(60.360, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(10, 'nm'),
    angle = ureg.Quantity(24.2, 'degree'),
    L = ureg.Quantity(2250, 'nm'))

sample1 = SampleModel1(
    cantilever = cantilever,
    h_s = ureg.Quantity(400, 'nm'),
    epsilon_s = ureg.Quantity(complex(20.0, -0.01), ''),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    sigma = ureg.Quantity(9.7e-7, 'S/cm'),
    rho = ureg.Quantity(1.9e15, '1/cm^3'),
    z_r = ureg.Quantity(1, 'nm'))

sample1_jit = SampleModel1Jit(**sample1.args())
h = ureg.Quantity(1000, 'nm')
wm = 1.0e5

Perform a finite-tip calculation.

In [7]:
obj['sphere'] = twodimCobject(sample1_jit)
obj['sphere'].addsphere(h, 20, 20)
obj['sphere'].set_alpha(1.0e-6)
obj['sphere'].set_breakpoints(15)
obj['sphere'].properties_dc()
obj['sphere'].properties_ac(omega_m=wm)
obj['sphere'].properties_am(omega_m=wm, omega_am = 250.)

obj['sphere'].print_key_results()

--------------------------------------------------
           Vts [V]       1.000    +1.000000e+00
             alpha       0.000    +1.000000e-06
       breakpoints      15.000    +1.500000e+01
           C0 [aF]       1.118    +1.118210e+00
         C1 [pF/m]      -0.005    -5.443494e-03
       C2 [mF/m^2]       0.000    +1.077103e-05
    gamma [pN s/m]       0.000    +2.935195e-05
   Delta f dc [Hz]      -0.000    -5.786213e-05
   Delta f ac [Hz]      -0.000    -2.880154e-05
   Delta f am [Hz]      -0.000    -1.080058e-05
--------------------------------------------------


The rms voltage error over the sphere surface is just 0.13 parts per million, indicating that the numerical solution is pretty good.

In [8]:
obj['sphere'].results['Vrms [ppm]']

0.1342943726022244

# Point-probe calculation

Mimic the code in `twodimCobject`.  In the point-probe approximation, the code can be comparatively much simpler, because the tip charge entering the friction and frequency-shift formulas is  

\begin{equation}
q_0 = C_0 V_{\mathrm{tip}}
\end{equation}

with 

\begin{equation}
C_0 = 4 \pi \epsilon_0 R
\end{equation}

and $R$ the tip radius.  Here $C_0$ is the capacitance of the spherical tip in the absence of the sample.

In [9]:
class experimentalobject():

    def __init__(self, sample):
        """Here sample is a SampleModel1Jit, SampleModel2Jit, SampleModel3Jit, or SampleModel4Jit object."""

        self.sample = sample

        self.Vr = ureg.Quantity(1, 'V')
        self.zr = ureg.Quantity(1, 'nm')

        self.results = {}
        self.results['Vts [V]'] = self.sample.cantilever.V_ts
        self.keys = ['Vts [V]']

        self.breakpoints = 10 

        self.results['breakpoints'] = self.breakpoints
        self.keys += ['breakpoints']
        
    def addsphere(self, h):
        """Model a sphere of radius $r$ above a ground plane, with a tip-sample
        separation of $h$.  Creates two unitless (1,3) arrays: (a) self.sj, a 
        voltage-test point at the center of the sphere, and (b) self.rk, 
        the location of the tip charge, also at the center of the sphere. 
        Coordinates are in nanometers.""" 
        
        r = ureg.Quantity(self.sample.cantilever.R, 'm')
        
        H = h.to('nm').magnitude
        R = r.to('nm').magnitude
        
        self.sj = np.array([[0., 0., H + R]])
        self.rk = np.array([[0., 0., H + R]])
        
        self.info = {'type': 'sphere', 
                     'r [nm]': R, 
                     'h [nm]': H}        
        
    def set_breakpoints(self, breakpoints):
        """Set the number of breakpoints to use in the numerical integration."""
        
        self.breakpoints = breakpoints 
        self.results['breakpoints'] = breakpoints    
        
    def solve(self, omega):
        """Compute the unitless K0, K1, and K2 integrals.  The units of the 
        integrals are 1/nm, 1/nm**2, and 1/nm**3 respectively.  The integrals
        are returned as pint quantities with units."""
        
        if self.sample.type == 4:
            
            K0, K1, K2 = KmatrixIV_jit(self.sj, self.rk)
            
        elif self.sample.type == 3:

            j0s = scipy.special.jn_zeros(0,100.)
            an, _ = scipy.integrate.newton_cotes(20, 1)
            
            args = {'omega': omega, 
                'omega0': self.sample.omega0,
                'kD': self.sample.kD, 
                'es': self.sample.epsilon_s, 
                'sj': self.sj, 
                'rk': self.rk, 
                'j0s': j0s, 
                'an': an,
                'breakpoints': self.breakpoints}
        
            K0, K1, K2 = KmatrixIII_jit(**args)

        elif self.sample.type == 2:

            j0s = scipy.special.jn_zeros(0,100.)
            an, _ = scipy.integrate.newton_cotes(20, 1)
            
            args = {'omega': omega, 
                'omega0': self.sample.omega0,
                'kD': self.sample.kD,
                'hd': self.sample.h_d,
                'ed': self.sample.epsilon_d,
                'es': self.sample.epsilon_s, 
                'sj': self.sj, 
                'rk': self.rk, 
                'j0s': j0s, 
                'an': an,
                'breakpoints': self.breakpoints}
        
            K0, K1, K2 = KmatrixII_jit(**args)

        elif self.sample.type == 1:

            j0s = scipy.special.jn_zeros(0,100.)
            an, _ = scipy.integrate.newton_cotes(20, 1)
            
            args = {'omega': omega, 
                'omega0': self.sample.omega0,
                'kD': self.sample.kD,
                'hs': self.sample.h_s,
                'es': self.sample.epsilon_s, 
                'ed': self.sample.epsilon_d,
                'sj': self.sj, 
                'rk': self.rk, 
                'j0s': j0s, 
                'an': an,
                'breakpoints': self.breakpoints}
        
            K0, K1, K2 = KmatrixI_jit(**args)

        else:

            raise Exception("unknown sample type")
        
        return ureg.Quantity(K0[0][0],'1/nm**1'), \
               ureg.Quantity(K1[0][0],'1/nm**2'), \
               ureg.Quantity(K2[0][0],'1/nm**3')
    
    def properties(self, omega_m):
        """Compute the cantilever friction and frequency shift when 
        DC and AC voltages are applied to the cantilever.  Here omega_m
        is the unitless voltage-modulation frequency."""
        
        K0dc, K1dc, K2dc = self.solve(0.)
        K0ac, K1ac, K2ac = self.solve(self.sample.cantilever.omega_c)

        V0 = ureg.Quantity(self.sample.cantilever.V_ts, 'V')
        fc = ureg.Quantity(self.sample.cantilever.f_c, 'Hz')
        kc = ureg.Quantity(self.sample.cantilever.k_c, 'N/m')        
        
        C0 = 4 * np.pi * epsilon0 * ureg.Quantity(self.sample.cantilever.R, 'm')
        q0 = C0 * V0
        
        wc_units = ureg.Quantity(self.sample.cantilever.omega_c, 'Hz')
        gamma = - (q0**2 * K2ac.imag)/(8 * np.pi * epsilon0 * wc_units)
        
        dK2ac = K2ac - K2dc
        Kterms = K2dc.real + 0.25 * dK2ac.real
        
        dfdc = - (fc * q0**2 * Kterms.real)/(4 * np.pi * epsilon0 * kc)
        
        wc = self.sample.cantilever.omega_c
        wp = wc + omega_m
        wm = wc - omega_m
        
        K0p, K1p, K2p = self.solve(np.abs(wp))
        K0m, K1m, K2m = self.solve(np.abs(wm))
        
        dK2p = K2p - K2dc
        dK2m = K2m - K2dc
        Kterms = K2ac + wc * 0.25 * (dK2p / wp + dK2m / wm)
        
        dfac = - (fc * q0**2 * Kterms.real)/(8 * np.pi * epsilon0 * kc)
        
        self.results['C0 [aF]'] = C0.to('aF').magnitude
        self.results['q0/qe'] = (q0/qe).to('').magnitude
        self.results['gamma [pN s/m]'] = gamma.to('pN s/m').magnitude
        self.results['Delta f dc [Hz]'] = dfdc.to('Hz').magnitude
        self.results['Delta f ac [Hz]'] = dfac.to('Hz').magnitude
        
        self.keys += ['C0 [aF]', 'q0/qe', 'gamma [pN s/m]']
        self.keys += ['Delta f dc [Hz]', 'Delta f ac [Hz]']
        
    def print_key_results(self):

        print('-'*50)
        for key in self.keys:
            print('{0:18s} {1:11.3f}    {1:+0.6e}'.format(
                key.rjust(18),
                self.results[key]))
        print('-'*50)

In [10]:
obj['point'] = experimentalobject(sample1_jit)
obj['point'].addsphere(h)
obj['point'].set_breakpoints(15)
obj['point'].properties(wm)
obj['point'].print_key_results()

--------------------------------------------------
           Vts [V]       1.000    +1.000000e+00
       breakpoints      15.000    +1.500000e+01
           C0 [aF]       1.113    +1.112650e+00
             q0/qe       6.945    +6.944615e+00
    gamma [pN s/m]       0.000    +2.896628e-05
   Delta f dc [Hz]      -0.000    -5.700749e-05
   Delta f ac [Hz]      -0.000    -2.804081e-05
--------------------------------------------------


Compare results to the spherical-tip results.

In [11]:
obj['sphere'].print_key_results()

--------------------------------------------------
           Vts [V]       1.000    +1.000000e+00
             alpha       0.000    +1.000000e-06
       breakpoints      15.000    +1.500000e+01
           C0 [aF]       1.118    +1.118210e+00
         C1 [pF/m]      -0.005    -5.443494e-03
       C2 [mF/m^2]       0.000    +1.077103e-05
    gamma [pN s/m]       0.000    +2.935195e-05
   Delta f dc [Hz]      -0.000    -5.786213e-05
   Delta f ac [Hz]      -0.000    -2.880154e-05
   Delta f am [Hz]      -0.000    -1.080058e-05
--------------------------------------------------


In [12]:
def compare_results(obj, key1, key2, keys):
    
    print('-'*71)
    print('{0:18s} {1:12s} {2:12s} {3:15s} {4:10s}'.format(
        'quantity'.rjust(18), 
        key1.rjust(12), 
        key2.rjust(12), 
        ('{:}/{:}'.format(key2, key1)).rjust(15),
        '|err| %'.rjust(10)))
    
    print('-'*71)
    for key in keys:
        val1 = obj[key1].results[key]
        val2 = obj[key2].results[key]

        print('{0:18s} {1:+12.4e} {2:+12.4e} {3:+15.4f} {4:10.2f}'.format(
            key.rjust(18),
            val1,
            val2,
            val2 / val1,
            100 * np.abs((val2 - val1)/val1)))

    print('-'*71)

# Conclusions

The sphere and point-probe calculation agree within a few percent.

To achieve this agreement, I had to make two changes.  First, I had to correct the point-probe formula for friction in the above code for `experimentalobject`; I am now using equation 30 for $\gamma_{\perp}$ in Loring's 2026-12-05 document.  Second, I had to fix the sign of the friction formula in `twodimCobject` in `dissipationtheory.dissipation13e`; now the friction comes out positive. 

In [13]:
compare_results(obj, 'sphere', 'point', 
    ['C0 [aF]', 'gamma [pN s/m]', 'Delta f dc [Hz]', 'Delta f ac [Hz]'])

-----------------------------------------------------------------------
          quantity       sphere        point    point/sphere    |err| %
-----------------------------------------------------------------------
           C0 [aF]  +1.1182e+00  +1.1127e+00         +0.9950       0.50
    gamma [pN s/m]  +2.9352e-05  +2.8966e-05         +0.9869       1.31
   Delta f dc [Hz]  -5.7862e-05  -5.7007e-05         +0.9852       1.48
   Delta f ac [Hz]  -2.8802e-05  -2.8041e-05         +0.9736       2.64
-----------------------------------------------------------------------


# Rerun using code in `dissipation17e.py`

I have moved the point-probe code into a separate file.

In [14]:
from dissipationtheory.dissipation17e import pointprobeCobject

Redo the above calculations using the object `pointprobeCobject` defined in `dissipation17e.py`.

In [15]:
obj['point 2'] = pointprobeCobject(sample1_jit)
obj['point 2'].addsphere(h)
obj['point 2'].set_breakpoints(15)
obj['point 2'].properties(wm)
obj['point 2'].print_key_results()

--------------------------------------------------
           Vts [V]       1.000    +1.000000e+00
       breakpoints      15.000    +1.500000e+01
           C0 [aF]       1.113    +1.112650e+00
             q0/qe       6.945    +6.944615e+00
    gamma [pN s/m]       0.000    +2.896628e-05
   Delta f dc [Hz]      -0.000    -5.700749e-05
   Delta f ac [Hz]      -0.000    -2.804081e-05
--------------------------------------------------


In [16]:
compare_results(obj, 'sphere', 'point 2', 
    ['C0 [aF]', 'gamma [pN s/m]', 'Delta f dc [Hz]', 'Delta f ac [Hz]'])

-----------------------------------------------------------------------
          quantity       sphere      point 2  point 2/sphere    |err| %
-----------------------------------------------------------------------
           C0 [aF]  +1.1182e+00  +1.1127e+00         +0.9950       0.50
    gamma [pN s/m]  +2.9352e-05  +2.8966e-05         +0.9869       1.31
   Delta f dc [Hz]  -5.7862e-05  -5.7007e-05         +0.9852       1.48
   Delta f ac [Hz]  -2.8802e-05  -2.8041e-05         +0.9736       2.64
-----------------------------------------------------------------------


::: {.content-hidden when-format="html"}

# Formatting notes

The header at the top of this file is for creating a nicely-formatted `.html` document using the program `quarto` ([link](https://quarto.org/)).  During development you can run the following command to render and display an `html` file on the fly.

    quarto preview dissipation-theory--Study-72.ipynb

When you are done developing this notebook, run  `quarto` from the command line as follows to create a nicely-formated `.html` version of the notebook.

    quarto render dissipation-theory--Study-72.ipynb && open dissipation-theory--Study-72.html
    

    
:::